# Attack Code Testing

## Imports

In [5]:
from adversary import *
from net import Net
import numpy as np
from torch.optim import Optimizer
import torch
from training import training, testing, accuracy
from optimizer import MiniBatchOptimizer
import matplotlib.pyplot as plt
from data_utils import get_mnist, build_data_loaders

## Setup of a Testing Net with Minibatch

In [6]:
accuracies_naive_fgsm = []
accuracies_naive_proj = []

criterion = torch.nn.CrossEntropyLoss()

epochs = 10
batch_size = 100
learning_rate = 0.01
decreasing_lr = False
use_cuda = True # GPU seems to raise erros on my side
device = torch.device('cuda' if use_cuda and torch.cuda.is_available() else 'cpu')


net_naive = Net().to(device)
train_dataset, test_dataset = get_mnist(normalize=True)
train_loader, test_loader = build_data_loaders(train_dataset, test_dataset, batch_size)

In [7]:
epsilons_fgsm = np.arange(0, 0.5, 0.05)

epsilons_proj = []

n = 784 # Thus n is the dimension of each image, which we need to scale the L2 norm for Projected Gradient Descent

# From source https://adversarial-ml-tutorial.org/adversarial_examples/ for L2 PGD
scaler = np.sqrt(2*n)/(np.sqrt(np.pi*np.exp(1)))
for eps in epsilons_fgsm:
    new_eps = eps*scaler
    epsilons_proj.append(new_eps)

## Train Naive Model

In [8]:
mini_opt = MiniBatchOptimizer(net_naive.parameters(), lr=learning_rate, decreasing_lr=decreasing_lr)
losses, acc = training(net_naive, train_loader, mini_opt, criterion, accuracy, epochs=epochs, device=device)
losses, acc = testing(net_naive, test_loader, criterion, accuracy, device=device)


Launching training on cuda
batch 100	loss = 1.956	acc = 0.62
batch 200	loss = 0.9253	acc = 0.76
batch 300	loss = 0.7016	acc = 0.76
batch 400	loss = 0.408	acc = 0.9
batch 500	loss = 0.3013	acc = 0.94
batch 600	loss = 0.3604	acc = 0.9
epoch 0	avg epoch loss = 0.8829	avg epoch acc = 0.7646
batch 100	loss = 0.1768	acc = 0.95
batch 200	loss = 0.1664	acc = 0.95
batch 300	loss = 0.3008	acc = 0.9
batch 400	loss = 0.2275	acc = 0.93
batch 500	loss = 0.1244	acc = 0.97
batch 600	loss = 0.2869	acc = 0.96
epoch 1	avg epoch loss = 0.207	avg epoch acc = 0.9404
batch 100	loss = 0.1056	acc = 0.97
batch 200	loss = 0.09397	acc = 0.96
batch 300	loss = 0.1809	acc = 0.94
batch 400	loss = 0.1422	acc = 0.94
batch 500	loss = 0.07289	acc = 0.99
batch 600	loss = 0.2617	acc = 0.98
epoch 2	avg epoch loss = 0.1393	avg epoch acc = 0.9593
batch 100	loss = 0.07156	acc = 0.98
batch 200	loss = 0.06446	acc = 0.97
batch 300	loss = 0.123	acc = 0.97
batch 400	loss = 0.09504	acc = 0.97
batch 500	loss = 0.05393	acc = 1.0
batch

## Attack Naive Model

Standard FGSM

In [9]:
losses, acc = attack(net_naive, criterion, accuracy, test_loader, epsilon=0.1, device=device)
print("Naive Net: ", acc)

Epsilon: 0.10	Test Accuracy = 0.873
Naive Net:  0.8725000000000003


In [10]:
for eps in epsilons_fgsm:
    loss_attack, acc_attack  = attack(net_naive, criterion, accuracy, test_loader, epsilon=eps, device=device)
    accuracies_naive_fgsm.append(acc_attack)

Epsilon: 0.00	Test Accuracy = 0.902
Epsilon: 0.05	Test Accuracy = 0.889
Epsilon: 0.10	Test Accuracy = 0.873
Epsilon: 0.15	Test Accuracy = 0.850
Epsilon: 0.20	Test Accuracy = 0.823
Epsilon: 0.25	Test Accuracy = 0.785
Epsilon: 0.30	Test Accuracy = 0.735
Epsilon: 0.35	Test Accuracy = 0.670
Epsilon: 0.40	Test Accuracy = 0.590
Epsilon: 0.45	Test Accuracy = 0.500


Projected Gradient Descent

In [11]:
epsilons_proj

[0.0,
 0.6775180286536014,
 1.355036057307203,
 2.0325540859608044,
 2.710072114614406,
 3.387590143268007,
 4.065108171921609,
 4.74262620057521,
 5.420144229228812,
 6.097662257882413]

In [12]:
for eps in epsilons_proj:
    loss_attack, acc_attack  = projected_attack(net_naive, criterion, accuracy, test_loader, epsilon=eps, alpha=0.1, num_iter=40, device=device)
    accuracies_naive_proj.append(acc_attack)


Epsilon: 0.00	Test Accuracy = 0.982
Epsilon: 0.68	Test Accuracy = 0.982
Epsilon: 1.36	Test Accuracy = 0.982
Epsilon: 2.03	Test Accuracy = 0.982
Epsilon: 2.71	Test Accuracy = 0.982
Epsilon: 3.39	Test Accuracy = 0.982
Epsilon: 4.07	Test Accuracy = 0.981
Epsilon: 4.74	Test Accuracy = 0.981
Epsilon: 5.42	Test Accuracy = 0.981
Epsilon: 6.10	Test Accuracy = 0.981


## Train Robust Models Version

In [ ]:
robust_net = Net().to(device)
protect_epochs = 10
protect_lr = 0.01
protect_bz = 32
protect_dec_lr = False

prot_train_loader, prot_test_loader = build_data_loaders(train_dataset, test_dataset, protect_bz)
mini_opt_proc = MiniBatchOptimizer(robust_net.parameters(), lr=protect_lr, decreasing_lr=protect_dec_lr)

accuracies_robust_fgsm = []
accuracies_robust_proj = []

In [ ]:
loss_train, acc_train = protected_training(robust_net, prot_train_loader, mini_opt_proc, criterion, accuracy, epochs=protect_epochs, device=device)
loss_test, acc_test = testing(robust_net, test_loader, criterion, accuracy, device=device)

## Attack Robust Model Version

Standard FGSM

In [ ]:
for eps in epsilons_fgsm:
    loss_attack, acc_attack  = attack(robust_net, criterion, accuracy, test_loader, epsilon=eps, device=device)
    accuracies_robust_fgsm.append(acc_attack)

Projected Gradient Descent

In [ ]:
for eps in epsilons_proj:
    loss_attack, acc_attack  = projected_attack(robust_net, criterion, accuracy, test_loader, epsilon=eps, alpha=1e-2, num_iter=40, device=device)
    accuracies_robust_proj.append(acc_attack)


## Comparative Analysis

In [ ]:
epsilons_it = np.arange(0, len(epsilons_pgd), 1)


### Naive Model

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(epsilons_it, accuracies_naive_fgsm, "*-", c='blue', label='Std. FGSM')
plt.plot(epsilons_it, accuracies_naive_proj, "*-", c='orange', label='PGD')

plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(epsilons_it)

plt.title("Naive Model")
plt.xlabel("Epsilon Choices")
plt.ylabel("Accuracy")
plt.legend();

### Robust Mode

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(epsilons_it, accuracies_robust_fgsm, "*-", c='blue', label='Std. FGSM')
plt.plot(epsilons_it, accuracies_robust_proj, "*-", c='orange', label='PGD')

plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(epsilons_it)

plt.title("Robust Model")
plt.xlabel("Epsilon Choices")
plt.ylabel("Accuracy")
plt.legend();

## File output

In [ ]:
print("********************************")
for entry in  accuracies_naive_fgsm:
    print(entry)
print("********************************")
for entry in  accuracies_naive_proj:
    print(entry)
print("********************************")
for entry in  accuracies_robust_fgsm:
    print(entry)
print("********************************")
for entry in  accuracies_robust_proj:
    print(entry)